In [1]:
import os
import xarray as xr
import geopandas as gpd  # type: ignore

from dotenv import load_dotenv
import sqlalchemy as sq
import sys
import pandas as pd
import matplotlib.pyplot as plt  # type: ignore

sys.path.append("../")
from Shared.DataService import DataService

/home/student/anghandk/.local/lib/python3.10/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_4093431/3748365637.py:3: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using Py

In [2]:
SOIL_GEOM_TABLE = "soil_geometry"
SOIL_COMP_TABLE = "soil_components"
SOIL_SURRONDINGS_TABLE = "soil_surronding_land"
SOIL_DATA_TABLE = "soil_data"

load_dotenv()
PG_DB = os.getenv("POSTGRES_DB")
PG_ADDR = os.getenv("POSTGRES_ADDR")
PG_PORT = os.getenv("POSTGRES_PORT")
PG_USER = os.getenv("POSTGRES_USER")
PG_PW = os.getenv("POSTGRES_PW")

In [3]:
if (
    PG_DB is None
    or PG_ADDR is None
    or PG_PORT is None
    or PG_USER is None
    or PG_PW is None
):
    raise ValueError("Environment variables not set")
db = DataService(PG_DB, PG_ADDR, int(PG_PORT), PG_USER, PG_PW)
conn = db.connect()

In [9]:
query = sq.text("select * FROM public.soil_data")
soil_data = pd.read_sql(query, conn)
soil_data.head()

,id,province,code,modifier,name,kind,water_table,root_restrict,restr_type,drainage,...,ksat,water_reten_0,water_reten_10,water_reten_33,water_reten_1500,bulk_density,elec_cond,calc_equiv,decomp_class,percnt_wood
0,MB$AR~~~~~N,MB,$AR,~~~~~,Acidic Bedrock,N,-,-,-,-,...,10.0,85,70,50,15,0.15,0,0,-9,0
1,MB$AR~~~~~N,MB,$AR,~~~~~,Acidic Bedrock,N,-,-,-,-,...,-9.0,-9,-9,-9,-9,2.80,-9,-9,-9,-9
2,MB$CB~~~~~N,MB,$CB,~~~~~,Cobble Beach,M,YU,0,-,I,...,60.0,34,6,5,1,1.80,0,20,-9,-9
3,MB$CB~~~~~N,MB,$CB,~~~~~,Cobble Beach,M,YU,0,-,I,...,50.0,37,6,5,1,1.70,0,20,-9,-9
4,MB$CB~~~~~N,MB,$CB,~~~~~,Cobble Beach,M,YU,0,-,I,...,25.0,40,6,5,1,1.60,0,15,-9,-9


In [10]:
soil_data = soil_data[["id", "province", "kind", "water_table", "root_restrict", "drainage", "parent_material_texture_1", "parent_material_chemical_1", "mode_of_depo_1", 'percnt_coarse_frag', 'sand_texture', 'total_sand', 'total_silt', 'total_clay', 'percnt_carbon', 'calcium_ph', 'proj_ph',  'water_reten_0', 'water_reten_10', 'water_reten_33',
       'water_reten_1500', 'bulk_density','elec_cond']]

In [11]:
soil_data

,id,province,kind,water_table,root_restrict,drainage,parent_material_texture_1,parent_material_chemical_1,mode_of_depo_1,percnt_coarse_frag,...,total_clay,percnt_carbon,calcium_ph,proj_ph,water_reten_0,water_reten_10,water_reten_33,water_reten_1500,bulk_density,elec_cond
0,MB$AR~~~~~N,MB,N,-,-,-,-,-,RKIA,0,...,-9,34.8,4.9,5.4,85,70,50,15,0.15,0
1,MB$AR~~~~~N,MB,N,-,-,-,-,-,RKIA,-9,...,-9,-9.0,-9.0,-9.0,-9,-9,-9,-9,2.80,-9
2,MB$CB~~~~~N,MB,M,YU,0,I,FR,UD,LACU,75,...,0,0.1,7.8,8.2,34,6,5,1,1.80,0
3,MB$CB~~~~~N,MB,M,YU,0,I,FR,UD,LACU,35,...,0,0.1,7.8,8.2,37,6,5,1,1.70,0
4,MB$CB~~~~~N,MB,M,YU,0,I,FR,UD,LACU,25,...,2,0.1,7.8,8.2,40,6,5,1,1.60,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36050,SKYKHS~~~~A,SK,M,YU,3,I,M,SA,TILL,0,...,23,1.1,7.7,7.8,47,-9,31,17,1.40,6
36051,SKYKHS~~~~A,SK,M,YU,3,I,M,SA,TILL,0,...,20,0.3,7.9,8.0,43,-9,28,16,1.50,6
36052,SKYKH~~~~~A,SK,M,YU,0,I,M,VC,TILL,0,...,20,4.3,7.4,7.5,55,-9,33,20,1.20,1
36053,SKYKH~~~~~A,SK,M,YU,0,I,M,VC,TILL,0,...,23,1.1,7.7,7.8,47,-9,31,17,1.40,1


In [12]:
query = sq.text("select * FROM public.soil_surronding_land")
surronding_soil = pd.read_sql(query, conn)
surronding_soil.head()

,poly_id,land_area,water_area,fresh_area,ocean_area,total_area
0,1001,844951,10923,718,10205,855874
1,1002,111030,0,0,0,111030
2,1003,46707,0,0,0,46707
3,1004,123419,0,0,0,123419
4,1005,185880,35167,389,34778,221047


In [13]:
surronding_soil = surronding_soil[["poly_id", "land_area"]]

In [14]:
surronding_soil.head()

,poly_id,land_area
0,1001,844951
1,1002,111030
2,1003,46707
3,1004,123419
4,1005,185880


In [15]:
query = sq.text("select * FROM public.soil_components")
soil_components = pd.read_sql(query, conn)
soil_components.head()

,poly_id,cmp,percent,slope,stone,surface_area,province,soil_code,modifier,profile,soil_id,coarse_frag_1,coarse_frag_2,depth,water_holding_cap
0,242021,1,40,A,U,D,AB,BUF,gl~~~,N,ABBUFgl~~~N,A,A,5,4
1,242021,2,30,A,N,U,AB,KEG,~~~~~,N,ABKEG~~~~~N,A,A,5,5
2,242021,3,30,A,N,F13,AB,MLD,~~~~~,N,ABMLD~~~~~N,A,A,5,5
3,242022,1,40,A,N,B07,AB,MKW,~~~~~,N,ABMKW~~~~~N,-,-,4,5
4,242022,2,30,A,N,F13,AB,MLD,~~~~~,N,ABMLD~~~~~N,A,A,5,5


In [16]:
soil_components = soil_components[["poly_id", "cmp", "percent", "province", "profile", "soil_id", "water_holding_cap"]]

In [17]:
soil_components.head()

,poly_id,cmp,percent,province,profile,soil_id,water_holding_cap
0,242021,1,40,AB,N,ABBUFgl~~~N,4
1,242021,2,30,AB,N,ABKEG~~~~~N,5
2,242021,3,30,AB,N,ABMLD~~~~~N,5
3,242022,1,40,AB,N,ABMKW~~~~~N,5
4,242022,2,30,AB,N,ABMLD~~~~~N,5


In [18]:
query = sq.text("select * FROM public.soil_geometry")
soil_geometry = pd.read_sql(query, conn)
soil_geometry.head()

,area,perimeter,poly_id,geometry
0,0.703365,12.299814,20002,0103000020130D0000010000009D00000094649194BB95...
1,1.448621,11.987536,20008,0103000020130D0000010000004F0000009E3180F0D918...
2,1.836356,19.000614,2002,0103000020130D000001000000790100009F4B3A656210...
3,0.801305,9.671116,20006,0103000020130D000001000000010100005AA7A6A97972...
4,0.515888,8.755135,16001,0103000020130D000001000000B4000000C2B7907EE88B...


In [19]:
db.cleanup()

In [20]:
soil_data.rename(columns={"id":"soil_id"}, inplace=True)

In [21]:
soil_data

,soil_id,province,kind,water_table,root_restrict,drainage,parent_material_texture_1,parent_material_chemical_1,mode_of_depo_1,percnt_coarse_frag,...,total_clay,percnt_carbon,calcium_ph,proj_ph,water_reten_0,water_reten_10,water_reten_33,water_reten_1500,bulk_density,elec_cond
0,MB$AR~~~~~N,MB,N,-,-,-,-,-,RKIA,0,...,-9,34.8,4.9,5.4,85,70,50,15,0.15,0
1,MB$AR~~~~~N,MB,N,-,-,-,-,-,RKIA,-9,...,-9,-9.0,-9.0,-9.0,-9,-9,-9,-9,2.80,-9
2,MB$CB~~~~~N,MB,M,YU,0,I,FR,UD,LACU,75,...,0,0.1,7.8,8.2,34,6,5,1,1.80,0
3,MB$CB~~~~~N,MB,M,YU,0,I,FR,UD,LACU,35,...,0,0.1,7.8,8.2,37,6,5,1,1.70,0
4,MB$CB~~~~~N,MB,M,YU,0,I,FR,UD,LACU,25,...,2,0.1,7.8,8.2,40,6,5,1,1.60,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36050,SKYKHS~~~~A,SK,M,YU,3,I,M,SA,TILL,0,...,23,1.1,7.7,7.8,47,-9,31,17,1.40,6
36051,SKYKHS~~~~A,SK,M,YU,3,I,M,SA,TILL,0,...,20,0.3,7.9,8.0,43,-9,28,16,1.50,6
36052,SKYKH~~~~~A,SK,M,YU,0,I,M,VC,TILL,0,...,20,4.3,7.4,7.5,55,-9,33,20,1.20,1
36053,SKYKH~~~~~A,SK,M,YU,0,I,M,VC,TILL,0,...,23,1.1,7.7,7.8,47,-9,31,17,1.40,1


In [22]:
merge_df = soil_components.merge(soil_data, on=["soil_id","province"], how="inner")

In [23]:
# merge_df = merge_df.merge(soil_geometry, on="poly_id", how="inner")

In [24]:
merge_df.head()

,poly_id,cmp,percent,province,profile,soil_id,water_holding_cap,kind,water_table,root_restrict,...,total_clay,percnt_carbon,calcium_ph,proj_ph,water_reten_0,water_reten_10,water_reten_33,water_reten_1500,bulk_density,elec_cond
0,242021,1,40,AB,N,ABBUFgl~~~N,4,M,YU,0,...,-9,31.1,4.4,4.6,85,65,-9,20,0.10,0
1,242021,1,40,AB,N,ABBUFgl~~~N,4,M,YU,0,...,6,1.4,4.2,4.8,51,-9,22,10,1.30,0
2,242021,1,40,AB,N,ABBUFgl~~~N,4,M,YU,0,...,20,0.5,4.6,5.0,42,-9,32,18,1.55,0
3,242021,1,40,AB,N,ABBUFgl~~~N,4,M,YU,0,...,22,0.3,6.0,6.5,42,-9,31,18,1.55,0
4,242021,1,40,AB,N,ABBUFgl~~~N,4,M,YU,0,...,21,0.0,5.4,5.6,43,-9,28,16,1.50,0


In [25]:
merge_df = merge_df.merge(surronding_soil, on="poly_id", how="inner")

In [26]:
merge_df.head()

,poly_id,cmp,percent,province,profile,soil_id,water_holding_cap,kind,water_table,root_restrict,...,percnt_carbon,calcium_ph,proj_ph,water_reten_0,water_reten_10,water_reten_33,water_reten_1500,bulk_density,elec_cond,land_area
0,242021,1,40,AB,N,ABBUFgl~~~N,4,M,YU,0,...,31.1,4.4,4.6,85,65,-9,20,0.10,0,118437
1,242021,1,40,AB,N,ABBUFgl~~~N,4,M,YU,0,...,1.4,4.2,4.8,51,-9,22,10,1.30,0,118437
2,242021,1,40,AB,N,ABBUFgl~~~N,4,M,YU,0,...,0.5,4.6,5.0,42,-9,32,18,1.55,0,118437
3,242021,1,40,AB,N,ABBUFgl~~~N,4,M,YU,0,...,0.3,6.0,6.5,42,-9,31,18,1.55,0,118437
4,242021,1,40,AB,N,ABBUFgl~~~N,4,M,YU,0,...,0.0,5.4,5.6,43,-9,28,16,1.50,0,118437


In [27]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40010 entries, 0 to 40009
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   poly_id                     40010 non-null  int64  
 1   cmp                         40010 non-null  int64  
 2   percent                     40010 non-null  int64  
 3   province                    40010 non-null  object 
 4   profile                     40010 non-null  object 
 5   soil_id                     40010 non-null  object 
 6   water_holding_cap           40010 non-null  object 
 7   kind                        40010 non-null  object 
 8   water_table                 40010 non-null  object 
 9   root_restrict               40010 non-null  object 
 10  drainage                    39968 non-null  object 
 11  parent_material_texture_1   39969 non-null  object 
 12  parent_material_chemical_1  39969 non-null  object 
 13  mode_of_depo_1              400

In [28]:
merge_df.duplicated().sum()

300

In [29]:
merge_df = merge_df.drop_duplicates()

In [30]:
merge_df.head()

,poly_id,cmp,percent,province,profile,soil_id,water_holding_cap,kind,water_table,root_restrict,...,percnt_carbon,calcium_ph,proj_ph,water_reten_0,water_reten_10,water_reten_33,water_reten_1500,bulk_density,elec_cond,land_area
0,242021,1,40,AB,N,ABBUFgl~~~N,4,M,YU,0,...,31.1,4.4,4.6,85,65,-9,20,0.10,0,118437
1,242021,1,40,AB,N,ABBUFgl~~~N,4,M,YU,0,...,1.4,4.2,4.8,51,-9,22,10,1.30,0,118437
2,242021,1,40,AB,N,ABBUFgl~~~N,4,M,YU,0,...,0.5,4.6,5.0,42,-9,32,18,1.55,0,118437
3,242021,1,40,AB,N,ABBUFgl~~~N,4,M,YU,0,...,0.3,6.0,6.5,42,-9,31,18,1.55,0,118437
4,242021,1,40,AB,N,ABBUFgl~~~N,4,M,YU,0,...,0.0,5.4,5.6,43,-9,28,16,1.50,0,118437


In [57]:
merge_df = pd.get_dummies(merge_df, columns=["profile", "kind", "water_table", 'root_restrict', 'drainage',
       'parent_material_texture_1', 'parent_material_chemical_1', 'mode_of_depo_1', 'sand_texture'])

In [58]:
merge_df.columns

Index(['poly_id', 'cmp', 'percent', 'province', 'soil_id', 'water_holding_cap',
       'percnt_coarse_frag', 'total_sand', 'total_silt', 'total_clay',
       'percnt_carbon', 'calcium_ph', 'proj_ph', 'water_reten_0',
       'water_reten_10', 'water_reten_33', 'water_reten_1500', 'bulk_density',
       'elec_cond', 'land_area', 'profile_A', 'profile_N', 'kind_M', 'kind_N',
       'kind_O', 'kind_U', 'water_table_-', 'water_table_NO', 'water_table_YB',
       'water_table_YG', 'water_table_YU', 'root_restrict_-',
       'root_restrict_0', 'root_restrict_1', 'root_restrict_2',
       'root_restrict_3', 'root_restrict_4', 'root_restrict_5',
       'root_restrict_6', 'root_restrict_7', 'root_restrict_8',
       'root_restrict_9', 'drainage_-', 'drainage_I', 'drainage_MW',
       'drainage_P', 'drainage_R', 'drainage_VP', 'drainage_VR', 'drainage_W',
       'parent_material_texture_1_-', 'parent_material_texture_1_C',
       'parent_material_texture_1_CS', 'parent_material_texture_1_F',
    

In [51]:
merge_df['water_holding_cap'] = pd.to_numeric(merge_df['water_holding_cap'], errors='coerce')

In [61]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39710 entries, 0 to 40009
Data columns (total 96 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   poly_id                        39710 non-null  int64  
 1   cmp                            39710 non-null  int64  
 2   percent                        39710 non-null  int64  
 3   province                       39710 non-null  object 
 4   soil_id                        39710 non-null  object 
 5   water_holding_cap              39572 non-null  float64
 6   percnt_coarse_frag             39710 non-null  int64  
 7   total_sand                     39710 non-null  int64  
 8   total_silt                     39710 non-null  int64  
 9   total_clay                     39710 non-null  int64  
 10  percnt_carbon                  39710 non-null  float64
 11  calcium_ph                     39710 non-null  float64
 12  proj_ph                        39710 non-null  floa

In [60]:
merge_df.groupby(['poly_id','province','soil_id']).agg('mean').reset_index()

,poly_id,province,soil_id,cmp,percent,water_holding_cap,percnt_coarse_frag,total_sand,total_silt,total_clay,...,mode_of_depo_1_RKUD,mode_of_depo_1_SPPT,mode_of_depo_1_TILL,mode_of_depo_1_UNDM,sand_texture_-,sand_texture_C,sand_texture_F,sand_texture_M,sand_texture_VC,sand_texture_VF
0,242021,AB,ABBUFgl~~~N,1.0,40.0,4.0,4.000000,34.0,28.600000,12.000000,...,0.0,0.0,1.0,0.0,0.20,0.0,0.000000,0.8,0.0,0.000000
1,242021,AB,ABKEG~~~~~N,2.0,30.0,5.0,0.000000,2.0,21.750000,44.500000,...,0.0,0.0,0.0,0.0,0.25,0.0,0.500000,0.0,0.0,0.250000
2,242021,AB,ABMLD~~~~~N,3.0,30.0,5.0,0.000000,-9.0,-9.000000,-9.000000,...,0.0,0.0,0.0,0.0,1.00,0.0,0.000000,0.0,0.0,0.000000
3,242022,AB,ABMKW~~~~~N,1.0,40.0,5.0,-9.000000,-9.0,-9.000000,-9.000000,...,0.0,1.0,0.0,0.0,1.00,0.0,0.000000,0.0,0.0,0.000000
4,242022,AB,ABMLD~~~~~N,2.0,30.0,5.0,0.000000,-9.0,-9.000000,-9.000000,...,0.0,0.0,0.0,0.0,1.00,0.0,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8638,9787003,AB,ABZGW~~~~~N,5.0,5.0,4.0,5.000000,40.0,33.333333,26.666667,...,0.0,0.0,0.0,1.0,0.00,0.0,0.000000,1.0,0.0,0.000000
8639,9787004,AB,ABHDY~~~~~A,3.0,15.0,4.0,3.666667,22.5,50.666667,26.833333,...,0.0,0.0,1.0,0.0,0.00,0.0,0.666667,0.0,0.0,0.333333
8640,9787004,AB,ABMAB~~~~~A,1.0,50.0,4.0,5.000000,36.6,35.600000,27.800000,...,0.0,0.0,1.0,0.0,0.00,0.0,1.000000,0.0,0.0,0.000000
8641,9787004,AB,ABROL~~~~~A,2.0,30.0,4.0,4.200000,40.2,34.200000,25.600000,...,0.0,0.0,1.0,0.0,0.00,0.0,0.000000,1.0,0.0,0.000000
